# Importing Libraries

In [4]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,SimpleRNN, GRU, Dropout,Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from keras.preprocessing.text import Tokenizer
import numpy as np
import math
import re
import time
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import re
import nltk
from contraction import CONTRACTION_MAP     # Its a py file contain expanded word of all short words like I'm
from bs4 import BeautifulSoup 

In [5]:
data = pd.read_csv("data_real.csv")
data.head()
data = data.replace(4,1)

# Cleaning tweets

In [6]:
# Data cleaning functions

def remove_htmltags(text):                      # Remove HTML tags
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

def remove_accented_chars(text):               # Normalizing accented charaters like ü
    import unicodedata
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP): # Expanding short words iike I've --> I have
    from contraction import CONTRACTION_MAP
    import contraction
    import re
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def remove_special_characters(text, remove_digits=False):              # Remove special characters
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def simple_stemmer(text):                                            # Stemming the words
    import nltk
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

def simple_lemmatize(text):                                          # lammetizing the words
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer() 
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

def remove_stopwords(text, is_lower_case=False):                    # Remove stopwords
    from nltk.corpus import stopwords
    from nltk.tokenize import WordPunctTokenizer
    tokenizer = WordPunctTokenizer()
    stopword_list =stopwords.words('english')
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def remove_hash_attherate(text):                                # Removing hastags and @ tags
    text = re.sub("#\w*", "",text)
    text = re.sub("@\w*", "",text)
    text = re.sub("\s+", " ", text)
    return text

def noramalize_text(document,htmltags = True, accented_chars = True, contractions_exp = True,
                   text_lower_case = True,special_characters = True, stemmer_text = True, 
                   lemmatize_text = True, stopwords_remove = False, remove_hash = True):
    
    normalized_doc = []
    
    for text in document:
        if htmltags:
            text = remove_htmltags(text)
        
        if accented_chars:
            text = remove_accented_chars(text)
        
        if contractions_exp:
            text = expand_contractions(text)
        
        if text_lower_case:
            text = text.lower()
        # remove extra line
        text = re.sub(r'[\r|\n|\r\n]+', ' ',text)
        
        if remove_hash:
            text = remove_hash_attherate(text)
            
        if special_characters:
            text = remove_special_characters(text)
            
        if stemmer_text:
            text = simple_stemmer(text)
        
        if lemmatize_text:
            text = simple_lemmatize(text)
        
        # remove extra whitespace
        text = re.sub(' +', ' ', text)   
        
        if stopwords_remove:
            text = remove_stopwords(text)
            
        normalized_doc.append(text)    
        
    return normalized_doc

In [7]:
data["cleaned"] = noramalize_text(data['tweets'],stemmer_text = False)
data.head()

c:\users\boltuzamaki\anaconda3\envs\pose\lib\site-packages\bs4\__init__.py:312: UserWarning: " i just received my G8 viola exam.. and its... well... .. disappointing.. :\.." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % self._decode_markup(markup)
c:\users\boltuzamaki\anaconda3\envs\pose\lib\site-packages\bs4\__init__.py:312: UserWarning: "E3 ON PLAYSTATION HOME IN ABOUT AN HOUR!!!!!!!!!! \../  \../" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % self._decode_markup(markup)


,tweets,labels,cleaned
0,is upset that he can't update his Facebook by ...,0,is upset that he cannot update his facebook by...
1,@Kenichan I dived many times for the ball. Man...,0,i dived many time for the ball managed to save...
2,my whole body feels itchy and like its on fire,0,my whole body feel itchy and like it on fire
3,"@nationwideclass no, it's not behaving at all....",0,no it is not behaving at all i am mad why am i...
4,@Kwesidei not the whole crew,0,not the whole crew


In [8]:
data.to_csv("cleaned.csv", index = None)
data = pd.read_csv("cleaned.csv")
text = data["cleaned"].astype(str)

# Tokenizing the tweets

In [9]:
MAX_LEN =50

def Tokenizing(data, tokenize_label = "", label = ""):
    # Spilitting in test and test data
    from sklearn.model_selection import train_test_split
    train, test = train_test_split(data, test_size=0.2)
    
    # Tokenizing using keras preprocessing library
    from keras.preprocessing.text import Tokenizer
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(text)
    sequences_train = tokenizer.texts_to_sequences(train["tweets"])
    
    sequences_test = tokenizer.texts_to_sequences(test["tweets"])
    
    vocab_size = len(tokenizer.word_index) + 1
    print(vocab_size)
    Y_train = train["labels"]
    Y_test = test["labels"]
    X_train = tf.keras.preprocessing.sequence.pad_sequences(sequences_train,
                                                           value = 0,
                                                           padding = 'post',
                                                           maxlen = MAX_LEN)
    X_test = tf.keras.preprocessing.sequence.pad_sequences(sequences_test,
                                                           value = 0,
                                                           padding = 'post',
                                                           maxlen = MAX_LEN)
    
    return X_test, X_train, Y_train, Y_test, tokenizer, vocab_size

In [10]:
X_test, X_train, Y_train, Y_test, tokenizer, vocab_size= Tokenizing(data, "tweets", "labels")

469399


# Saving the tokenizer for future use

In [11]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Creating the model

In [12]:
num_word = vocab_size
input_length = 50
dropout_val = 0.2
Dense_layers = 256
RNN_layer = 128
no_of_embeddings = 100
max_pad_length = 50

In [13]:
def model_RNN(layer = "RNN"):
    if layer == "RNN":
        model = Sequential()
        model.add(Embedding(num_word +1 ,  no_of_embeddings ,input_length=max_pad_length))
       # model.add(SimpleRNN(RNN_layer,return_sequences=True))
       # model.add(Dropout(dropout_val))
        model.add(SimpleRNN(RNN_layer))
        model.add(Dropout(dropout_val))
        model.add(Dense(Dense_layers, activation = 'relu'))
        model.add(Dropout(dropout_val))
        model.add(Dense(1 , activation = 'sigmoid'))
    
    if layer == "LSTM":
        model = Sequential()
        model.add(Embedding(num_word +1 ,  no_of_embeddings ,input_length=max_pad_length))
        model.add(LSTM(RNN_layer,return_sequences=True))
        model.add(Dropout(dropout_val))
        model.add(LSTM(RNN_layer))
        model.add(Dropout(dropout_val))
        model.add(Dense(Dense_layers, activation = 'relu'))
        model.add(Dropout(dropout_val))
        model.add(Dense(1 , activation = 'sigmoid'))
        
    if layer == "GRU":
        model = Sequential()
        model.add(Embedding(num_word +1 ,  no_of_embeddings ,input_length=max_pad_length))
        model.add(GRU(RNN_layer,return_sequences=True))
        model.add(Dropout(dropout_val))
        model.add(GRU(RNN_layer))
        model.add(Dropout(dropout_val))
        model.add(Dense(Dense_layers, activation = 'relu'))
        model.add(Dropout(dropout_val))
        model.add(Dense(1 , activation = 'sigmoid'))
        
    if layer == "BILSTM":
        model = Sequential()
        model.add(Embedding(num_word +1 ,  no_of_embeddings ,input_length=max_pad_length))
        model.add(Bidirectional(LSTM(RNN_layer,return_sequences=True)))
        model.add(Dropout(dropout_val))
        model.add(Bidirectional(LSTM(RNN_layer)))
        model.add(Dropout(dropout_val))
        model.add(Dense(Dense_layers, activation = 'relu'))
        model.add(Dropout(dropout_val))
        model.add(Dense(1 , activation = 'sigmoid'))
        
    return model

# Compiling and training the model

In [14]:
mod = model_RNN(layer = "RNN")        # Choose layers anyone from LSTM, GRU , BILSTM , and simple RNN

In [ ]:
mod.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
mod.fit(X_train, Y_train, batch_size =Batch_size, epochs = 2 , verbose = 1)

# Saving and evaluating the model

In [ ]:
mod.save("model.h5")

In [ ]:
mod.evaluate(X_test ,Y_test, batch_size =Batch_size )

# Loading the tokenizer pickle and test it on any sentence

In [ ]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
text = ["This movie is not so good", "This suppose to be bad but it is good"]

In [ ]:
sequences_test = tokenizer.texts_to_sequences(text)
test = tf.keras.preprocessing.sequence.pad_sequences(sequences_test,value = 0,padding = 'post', maxlen = MAX_LEN)

In [ ]:
pred = mod.predict(test)

In [ ]:
sentiment_list = []

for i in range(0,pred.shape[0]):
    if pred[i]>0.5:
        value = "Positive"
    if pred[i]<=0.5:
        value = "Negative"
    sentiment_list.append(value) 

In [ ]:
sentiment_list